In [ ]:
#python3 t2_split_gs_score.py --cfv 2 --fromf ts2_16x260_inv

In [ ]:
#splits cancerous data among gleason score subfolders in trainf and validf folders

In [1]:
import os, gc, subprocess, time, sys, shutil, argparse

import scipy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm import tqdm
#from tqdm.notebook import tqdm
import skimage.io
from skimage.transform import resize, rescale
from math import sqrt
import statistics

sys.path.append("..")

In [2]:
import PIL.Image
#PIL.Image.MAX_IMAGE_PIXELS = 400000000
PIL.Image.MAX_IMAGE_PIXELS = None

In [3]:
from panda_bvv_config import *

## 1. Inputs ##

In [8]:
Zoom = 2

#these 2 values are for x5 cross-fold validation:
cfv_const= -2101
last_train_label = 7500
last_valid_label = 9300


In [5]:
ap = argparse.ArgumentParser()
ap.add_argument('--cfv', help="cross fold validation number", dest = 'cfv', default = 1,\
                type=int, choices=choices_)
ap.add_argument('--fromf', help="from folder name", dest = 'fromf',\
                type=str)
                #("--cnn", type=str, help="training cnn name", dest = 'train_cnn')
args = vars(ap.parse_args())
#data-from folder:
new_proc_folder = args["fromf"]
cfv = args["cfv"]

In [6]:
# test:
# cfv_const = -400
# new_proc_folder = cancer_s2
# cfv = 2
# input_df = test_cnn_labels

In [7]:
# wrapping inputs:
new_proc_folder = os.path.join(base_path, new_proc_folder)


## 2. Standard functions ##

## 5. Distribute processed files among training and validation folders ##

In [7]:
try:
    shutil.rmtree(train_cnn)
    shutil.rmtree(valid_cnn)
except OSError as ee:
    print ("Error: %s: " % ee)
os.mkdir(train_cnn)
os.mkdir(valid_cnn)
for score in gs_scores:
    os.mkdir(os.path.join(train_cnn, f'{score}'))
    os.mkdir(os.path.join(valid_cnn, f'{score}'))
# 

In [9]:
#cross-fold validation:
labels_perm = mask_labels.sample(frac = 1, random_state=npseed)
if cfv == 0: #take data without test:    
    train_cnn_labels = labels_perm[:last_train_label]
    valid_cnn_labels = labels_perm[last_train_label:last_valid_label]
elif cfv == 1:
    train_cnn_labels = labels_perm[:cfv_const]
    valid_cnn_labels = labels_perm[cfv_const:]
else:
    train_cnn_labels = pd.concat([labels_perm[:cfv_const*cfv], labels_perm[cfv_const*(cfv-1):]], sort=False)
    valid_cnn_labels = labels_perm[cfv_const*cfv:cfv_const*(cfv-1)]

In [48]:
#split train files among ISUP categories
with tqdm(total=train_cnn_labels.shape[0]) as pbar:
    for j, row in train_cnn_labels.iterrows():
        try:
            filename = row['image_id'] + '.png'
            suffix = row['gleason_score'].replace('+', '_') if row['gleason_score'] != '5+3' else '3_5'            
            path_to_train_class = os.path.join(train_cnn, suffix, filename)
            path_to_copy_from = os.path.join(new_proc_folder, filename)
            bashCommand = "cp " + path_to_copy_from + ' ' + path_to_train_class
            process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
            output, error = process.communicate()
            if error: print(str(error))
        except Exception as ee:
            print(f'Image number {filename}, error:\n{ee}')
        pbar.update(1)

In [27]:
#split validation files among ISUP categories
with tqdm(total=valid_cnn_labels.shape[0]) as pbar:
    for j, row in valid_cnn_labels.iterrows():
        try:
            filename = row['image_id'] + '.png'
            suffix = row['gleason_score'].replace('+', '_') if row['gleason_score'] != '5+3' else '3_5'     
            path_to_valid_class = os.path.join(valid_cnn, suffix, filename)    
            path_to_copy_from = os.path.join(new_proc_folder, filename)
            bashCommand = "cp " + path_to_copy_from + ' ' + path_to_valid_class    
            process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
            output, error = process.communicate()
            if error: print(str(error))
        except Exception as ee:
            print(f'Image number {filename}, error:\n{ee}')
        pbar.update(1)

100%|██████████| 400/400 [00:56<00:00,  7.14it/s]


In [1]:
import os
module_name = 't2_split_gs_score'

os.system('jupyter nbconvert --to python ' + module_name + '.ipynb')
with open(module_name + '.py', 'r') as f:
    lines = f.readlines()
with open(module_name + '.py', 'w') as f:
    for line in lines:
        if 'nbconvert --to python' in line:
            break
        else:
            f.write(line)